# **Make a copy of this python notebook**

# Library Installation

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e
!pip install --pre deepchem


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3457  100  3457    0     0   8909      0 --:--:-- --:--:-- --:--:--  8909


add /root/miniconda/lib/python3.7/site-packages to PYTHONPATH
python version: 3.7.12
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing openmm, pdbfixer
added conda-forge to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda

     |████████████████████████████████| 608 kB 4.2 MB/s 
     |████████████████████████████████| 20.6 MB 6.9 MB/s 


In [ ]:
pip install dgl


     |████████████████████████████████| 4.4 MB 4.1 MB/s 


In [ ]:
pip install dgllife

     |████████████████████████████████| 133 kB 4.2 MB/s 
  Created wheel for dgllife: filename=dgllife-0.2.8-py3-none-any.whl size=213162 sha256=7f681e5f35332637144cd36752b903ce59c3ef86d691ace1c02b358a74bef465
  Stored in directory: /root/.cache/pip/wheels/b6/81/6f/6bbc4b7a80b06d92829b5362a78277173c9e7b46d294d4ca45
Successfully built dgllife


In [ ]:

import pandas as pd
import deepchem as dc
from PIL import Image
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from deepchem.models import GCNModel
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from itertools import cycle
from scipy import interp
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
# import init_db as database
import sys
import os
from pathlib import Path
import numpy as np

# Mount your drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Model Information

In [ ]:
#Enter the model name in mode section
mode = 'Select_Model' # mode ={"gcn": Graph Convolution Model,"gat": Graph Attentive Network ,"attentive": Attentive FP,"dag": Directed Acyclic Graph}

# Enter the path of query data
data_path = 'Path of the the query data'

# Enter the path of the directory where the model checkpoint is saved
path_checkpoint = 'Directory path of the checkpoint'

# deepGraphh

In [ ]:
# run this section to get the results for query data
##*************** Please do not make any changes in this section****************##

data = pd.read_csv(data_path )

if mode == 'gcn':
  featurizer = dc.feat.MolGraphConvFeaturizer()
  data = data[['SMILES', 'Activation Status']]
  ret = featurizer.featurize(data['SMILES'])
  indexes_notF = list()
  for i in range(0, len(data)):
    if not ret[i]:
      indexes_notF.append(i)
  data = data.drop(indexes_notF)
  with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:
    data.to_csv(tmpfile.name)
    loader = dc.data.CSVLoader(["Activation Status"], feature_field="SMILES",
                                featurizer=dc.feat.MolGraphConvFeaturizer())
    dataset = loader.create_dataset(tmpfile.name)
  model = dc.models.GCNModel(mode='classification', n_tasks=1, batch_size=32, learning_rate=0.001,n_classes=2,model_dir=path_checkpoint)

  model.restore(model_dir=path_checkpoint)
  user_predict_data = model.predict(dataset)
  pred = np.array(user_predict_data)
  for i in range(0, len(pred[0])):
    ithcol = pred[:, i]
    stringClass = 'class ' + str(i)
    data[stringClass] = ithcol
  data.to_csv('Input_data_Prob_Matrix.csv')


if mode == 'gat':
    featurizer = dc.feat.MolGraphConvFeaturizer()
    data = data[['SMILES', 'Activation Status']]
    ret = featurizer.featurize(data['SMILES'])
    indexes_notF = list()
    for i in range(0, len(data)):
      if not ret[i]:
        indexes_notF.append(i)
    data = data.drop(indexes_notF)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:
      data.to_csv(tmpfile.name)
      loader = dc.data.CSVLoader(["Activation Status"], feature_field="SMILES",
                                 featurizer=dc.feat.MolGraphConvFeaturizer())
      dataset = loader.create_dataset(tmpfile.name)
    
    model = dc.models.GATModel(mode='classification', n_tasks=1, batch_size=32, learning_rate=0.001,n_classes = 2,model_dir=path_checkpoint)

    model.restore(model_dir=path_checkpoint)
    user_predict_data = model.predict(dataset)
    pred = np.array(user_predict_data)
    for i in range(0, len(pred[0])):
      ithcol = pred[:, i]
      stringClass = 'class ' + str(i)
      data[stringClass] = ithcol
    data.to_csv('Input_data_Prob_Matrix.csv')

if mode == 'attentive':
  featurizer = dc.feat.MolGraphConvFeaturizer()
  data = data[['SMILES', 'Activation Status']]
  ret = featurizer.featurize(data['SMILES'])
  indexes_notF = list()
  for i in range(0, len(data)):
    if not ret[i]:
      indexes_notF.append(i)
  data = data.drop(indexes_notF)
  with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:
    data.to_csv(tmpfile.name)
    loader = dc.data.CSVLoader(["Activation Status"], feature_field="SMILES",
                                featurizer=dc.feat.MolGraphConvFeaturizer(use_edges=True))
    dataset = loader.create_dataset(tmpfile.name)
  
  model = dc.models.AttentiveFPModel(mode='classification', n_tasks=1, batch_size=32, learning_rate=0.001,n_classes = 2,model_dir=path_checkpoint)

  model.restore(model_dir=path_checkpoint)
  user_predict_data = model.predict(dataset)
  pred = np.array(user_predict_data)
  for i in range(0, len(pred[0])):
    ithcol = pred[:, i]
    stringClass = 'class ' + str(i)
    data[stringClass] = ithcol
  data.to_csv('Input_data_Prob_Matrix.csv')


if mode == 'dag':
  featurizer = dc.feat.ConvMolFeaturizer()
  data = data[['SMILES', 'Activation Status']]
  ret = featurizer.featurize(data['SMILES'])
  indexes_notF = list()
  for i in range(0, len(data)):
    if not ret[i]:
      indexes_notF.append(i)
  data = data.drop(indexes_notF)
  with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:
    data.to_csv(tmpfile.name)
    loader = dc.data.CSVLoader(["Activation Status"], feature_field="SMILES",
                                featurizer=dc.feat.ConvMolFeaturizer())
    dataset = loader.create_dataset(tmpfile.name)
  trans = dc.trans.DAGTransformer(max_atoms=50)
  dataset = trans.transform(dataset)

  model = dc.models.DAGModel(mode='classification', n_tasks=1, batch_size=32, learning_rate=0.001,n_classes = 2,model_dir=path_checkpoint)

  model.restore(model_dir=path_checkpoint)
  user_predict_data = model.predict(dataset)
  pred = np.array(user_predict_data)
  for i in range(0, len(pred[0][0])):
    ithcol = list()
    for k in range(0, len(pred)):
      ithcol.append(pred[k][0][i])
    stringClass = 'class ' + str(i)
    data[stringClass] = ithcol
  data.to_csv('Input_data_Prob_Matrix.csv')
  ## Result will be saved in the same folder as path_checkpoint, user can change the path if necessary.